In [1]:
import numpy as np
import pandas as pd
import json
import os
import math

from disorder.disorder import Disorder
from disorder.entropy import Entropy
from disorder.cifreader import Read_CIF
from pymatgen.core.composition import Composition
from pymatgen.core.periodic_table import Species

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
elem_list=['Ac', 'Ag', 'Al', 'Am', 'As', 'At', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Bk', 'Br', 'C', 'Ca',\
 'Cd', 'Ce', 'Cf', 'Cl', 'Cm', 'Co', 'Cr', 'Cs', 'Cu', 'Dy', 'Er', 'Eu', 'F', 'Fe',\
 'Fr', 'Ga', 'Gd', 'Ge','H', 'Hf', 'Hg', 'Ho', 'I', 'In', 'Ir', 'K', 'La', 'Li', 'Lu',\
 'Mg', 'Mn', 'Mo', 'N', 'Na', 'Nb', 'Nd', 'Ni', 'No', 'Np', 'O', 'Os', 'P', 'Pa',\
 'Pb', 'Pd', 'Pm', 'Po', 'Pr', 'Pt', 'Pu', 'Ra', 'Rb', 'Re', 'Rh', 'Ru', 'S', 'Sb', 'Sc',\
 'Se', 'Si', 'Sm', 'Sn', 'Sr', 'Ta', 'Tb', 'Tc', 'Te', 'Th', 'Ti', 'Tl', 'Tm', 'U', 'V',\
 'W', 'Xe', 'Y', 'Yb', 'Zn', 'Zr']

In [3]:
path='path to the forlder with CIFs'
data=pd.read_csv('calculated_data/H_files.csv')

In [4]:
list_of_files=data['id'].values
len(list_of_files)

38603

In [ ]:
from tqdm import tqdm
from time import time

disordered_comp={}
exc=[]
exc_large=[]
exc_elem=[]
exc_form=[]
errors=[]
errors_large=[]

t1=time()
limit_low=0
limit_high=len(list_of_files)
for i in tqdm(range(limit_low,limit_high)):
    file=list_of_files[i]
    el_switch=0
    form_switch=0
    try:
        cif=Read_CIF(file=path+file)
        composition=Composition(cif.read_formula).as_dict()
        for el in composition.keys():
            if(el not in elem_list):
                exc_elem.append(file)
                el_switch=1
                break
                
        o=cif.orbits()
        form_el=composition.keys()
        struct_el=[]
        for name in o['atom_site_type_symbol'].values:
            struct_el.append(str(Species(name).element))
        struct_el=set(struct_el)
        if(('H' in form_el) and ('H' not in struct_el)):
            struct_el=struct_el|{'H'}
        if(('D' in form_el) and ('D' not in struct_el)):
            struct_el=struct_el|{'D'}
        if(struct_el!=form_el):
            form_switch=1
            exc_form.append(file)
                
        if(el_switch==0 and form_switch==0):
            dis=Disorder(file=path+file, radius_file='data/all_radii.csv')
            compound={}
            compound['formula']=dis.material.read_formula
            compound['ICSD_ID']=dis.material.read_id
            compound['group_num']=dis.material.space_group
            compound['Z']=dis.material.z

            # print(VPorbits,nan_value_rad)
            if(len(dis.positions)<501):
                orbits=dis.classify()
                compound=compound|orbits.to_dict()
                ent=Entropy(file=path+file,radius_file='data/all_radii.csv')
                compound['mixing_entropy']=ent.calculate_entropy('mixing')
                compound['conf_entropy']=ent.calculate_entropy('configurational')
                disordered_comp[file]=compound
            else:
                exc_large.append(file)
                errors_large.append(list(set(dis.return_errors())))
                                    
    except:
        exc.append(file)
        errors.append(list(set(dis.return_errors())))
        
t2=time()
print(t2-t1)

 62%|██████▏   | 6239/10000 [9:20:34<48:42,  1.29it/s]     /Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:638: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
/Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:640: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
/Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:822: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
/Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:824: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
 64%|██████▍   | 6419/10000 [9:46:35<3:23:09,  3.40s/

 80%|███████▉  | 7975/10000 [12:24:02<2:33:59,  4.56s/it] /Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:638: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
/Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:640: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
/Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:822: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
/Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:824: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
 84%|████████▎ | 8355/10000 [13:14:12<3:15:54,  7.15s/

 95%|█████████▌| 9508/10000 [14:18:25<04:07,  1.99it/s]  /Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:638: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
/Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:640: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
/Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:822: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
/Users/elenapatyukova/Documents/GitHub/Disorder/disorder.py:824: RuntimeWarning: invalid value encountered in log
  (1-self.data['internal_intersect_index'][i][0]*j)*np.log(1-j*self.data['internal_intersect_index'][i][0])
 97%|█████████▋| 9705/10000 [14:32:17<38:49,  7.90s/it]

53473.58637905121


In [ ]:
print(len(disordered_comp))
print(len(exc_elem))
print(len(exc_large))
print(len(exc))

In [ ]:
with open('H_disorder_results.json', 'w') as fp:
    json.dump(disordered_comp, fp, indent=4)